In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
from collections import defaultdict

In [7]:
state_districtsDf = pd.read_csv('districts_updated.csv', index_col=None)

In [92]:
folder='CSR Year 2016-17'

In [23]:
def get_csr_data(state, district, year='FY 2016-17'):
    print(state, district)
    url = 'https://csr.gov.in/CSR/companylist.php?compCat=&year={}&district={}&state={}&mact=&csr_spent_range='.format(year, district, state)
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    amount = 0
    data = []
    rows = soup.findAll('tr')
    for row in rows[1:]:
        row_data = []
        columns = row.findAll('td')
        if len(columns) != 3:
            continue
        for col in columns:
            row_data.append(col.text.strip())
        data.append(row_data)
        amount += float(row_data[-1].replace(',',''))
    return data,amount

In [33]:

for i in range(state_districtsDf.shape[0]):
    total_csr_spent = {}
    temp = list(state_districtsDf.iloc[i,:])
    state = temp[0]
    districts = []
    os.system('mkdir "{}/{}"'.format(folder,state))
    district_csr_spent = {} 
    for district in temp[1:]:
        if pd.isna(district):
            break
        data, amount = get_csr_data(state, district)
        district_details_df = pd.DataFrame(data, columns=['SNo.', 'company', 'amount'])
        district_details_df.to_csv(folder + '/' + state + '/' + district + '.csv')
        district_csr_spent[district] = amount
    total_csr_spent[state] = district_csr_spent 
    total_df = pd.DataFrame.from_dict(total_csr_spent)
    total_df.to_csv(folder + '/' + state + '/' + 'Z_total.csv')

def get_companies(folder_name):
    companies = defaultdict(int)
    for i in range(state_districtsDf.shape[0]):
        state = state_districtsDf.iloc[i][0]
        districts = list(state_districtsDf.iloc[i][1:])
        for district in districts:
            if pd.isna(district):
                break
            file_name = folder_name + '/' + state + '/' + district + '.csv'
            dataFrame = pd.read_csv(file_name, usecols=['company', 'amount'])#, usecols=['company', 'amount']) #2:company name, 3:amount
            for i in range(dataFrame.shape[0]):
                companies[dataFrame.loc[i,'company']] += float(str(dataFrame.loc[i, 'amount']).replace(',',''))
    return companies

In [51]:
companies_invested_in_csr = get_companies(folder)

companies_invested_in_csrDf = pd.DataFrame.from_dict(companies_invested_in_csr, orient='index', dtype=int)

companies_invested_in_csrDf.sort_values(by=0, ascending = False, inplace=True)

In [52]:
companies_invested_in_csrDf.to_csv(folder + '/companies_with_amount.csv')

### Need to be fetched once only

from py2neo import Graph
graph = Graph()

query = 'match(c:company) return c.name as name, c.cin as CIN'
result = graph.data(query)

companies_df = pd.DataFrame.from_records(result)

companies_df.to_csv('companies_in_graph_db.csv')

In [15]:
companies_df = pd.read_csv('./companies_in_graph_db.csv')

In [17]:
companies_invested_in_csr_df.head()

,name
0,3D Plm Software Solutions Limited
1,A K Capital Services Limited
2,A L M Industries Limited
3,A R B Bearings Limited
4,A V Thomas Leather & Allied Products Private L...


In [18]:
companies_df.head()

,Unnamed: 0,CIN,name
0,0,L00000CH1983PLC031318,SAB INDUSTRIES LIMITED
1,1,L00000CH1990PLC010573,LAKSHMI ENERGY AND FOODS LIMITED
2,2,L00000HP1994PLC014770,HITKARI INDUSTRIES LIMITED
3,3,L00000MH1946PLC197474,QUADRANT TELEVENTURES LIMITED
4,4,L01100MH1996PLC100380,USHER AGRO LIMITED


In [19]:
companies_df.sort_values(by='name', inplace=True)

In [21]:
companies_df = companies_df.reset_index()
companies_df = companies_df.drop(labels = ['index'], axis=1)

In [56]:
tempDf = companies_df.merge(companies_invested_in_csr_df, on='name', how='inner')

In [93]:
companies_invested_in_csrDf = pd.read_csv(folder + '/companies_with_amount.csv', index_col=False)

companies_without_df = pd.read_csv(folder + '/companies.csv')

companies_invested_in_csrDf.columns=['name','amount']

finalDf = companies_without_df.merge(companies_invested_in_csrDf, how='inner', on='name')

finalDf.to_csv(folder + '/companies.csv')